In [ ]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
import joblib
import pickle
import requests
import json

: 

In [ ]:
# 1. Load dataset
df = pd.read_csv('data_balita.csv')
print(df.head())

In [8]:
# 2. Preprocessing data
# mengkodekan jenis kelamin
df['Jenis Kelamin'] = df['Jenis Kelamin'].map({'laki-laki': 0, 'perempuan': 1})
# mengkodekan status gizi
df['Status Gizi'] = df['Status Gizi'].map({'severely stunted': 'stunting', 'stunted': 'stunting', 'normal': 'normal', 'tinggi': 'tinggi'})


In [ ]:
# 3. Visualisasi data
sns.countplot(x='Status Gizi', data=df)
plt.title('Distribusi Status Gizi')
plt.show()

sns.pairplot(df, hue='Status Gizi')
plt.show()

In [10]:
# 4. Memisahkan fitur dan target
X = df.drop('Status Gizi', axis=1)
y = df['Status Gizi']

In [11]:
# Membagi dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 5. Memodelkan model KNN
# membuat objek KNN dengan k=3
knn = KNeighborsClassifier(algorithm='kd_tree', n_neighbors=3)
# melatih model KNN dengan data latih
knn.fit(X_train, y_train)

In [ ]:
# 6. Mengevaluasi model
# membuat prediksi dengan data uji
y_pred = knn.predict(X_test)

# menghitung akurasi model
accuracy = accuracy_score(y_test, y_pred)
print('Akurasi model KNN:', accuracy)

# Menampilkan confusion matrix dan classification report
print('Confusion Matrix:')
cm = confusion_matrix(y_test, y_pred)
print(cm)

print('Classification Report:')
print(classification_report(y_test, y_pred))

In [ ]:
# Visualisasi confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['stunting', 'normal', 'tinggi'], yticklabels=['stunting', 'normal', 'tinggi'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Cross-validation
cross_val_scores = cross_val_score(knn, X, y, cv=5)
print('Cross-validation scores:', cross_val_scores)
print('Mean cross-validation accuracy:', np.mean(cross_val_scores))

In [ ]:
joblib.dump(knn, "model.pkl")
print("Model telah disimpan sebagai model.pkl")

In [16]:
url = "http://127.0.0.1:8000/predict"

In [17]:
def kirim_permintaan(data):
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, data=json.dumps(data), headers=headers)
    return response.json()

In [18]:
data_contoh = {
    "umur": 20,
    "jenis_kelamin": "perempuan",
    "tinggi_badan": 77.0
}

In [ ]:
hasil = kirim_permintaan(data_contoh)
print("Hasil Prediksi:", hasil)

In [ ]:
# 7. Test / Simulasi
test = pd.DataFrame({'Umur (bulan)': [20], 'Jenis Kelamin': [1], 'Tinggi Badan (cm)': [77]})
pred = knn.predict(test)
print("Hasil Tes = {}".format(np.array(pred)))

In [ ]:
import pickle 

filename = 'data_balita.sav'
pickle.dump(knn, open(filename, 'wb'))